# We want to create a synthetic time series data set

> Based on the synthetic data generation notebook of the ```afa``` repository by Henrik von Kleist.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pathlib
relative_path_to_afa = '../../afa_github/afa'
#sys.path.insert(0, os.path.abspath(relative_path_to_afa))
sys.path.insert(1, os.path.join(sys.path[0], relative_path_to_afa))
print('CWD:', pathlib.Path(pathlib.Path.cwd()))

import sklearn

# data generation
from afa_datasets.synthetic_data.data_generation_ts import generate_data_ts

# loading/ saving
from data_modelling.clairvoyance_static.datasets.data_loader import CSVLoader, save_dataset
from data_modelling.clairvoyance_static.utils.model_utils import PipelineComposer

CWD: /home2/joshua.wendland/Documents/sepsis/toy_dataset


2022-11-28 12:47:56.559341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 12:47:56.559367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# define directories
data_dir = "dataset/"
dataset_name = "synthetic_1"

# Datageneration

Use sklearn's make-classification function

In [26]:
# df_train, df_test = generate_data_ts(data_dir = data_dir + dataset_name + "/", 
#                                          dataset_name = dataset_name,
#                                          n_features_static  = 4 ,
#                                          n_features_ts  = 10 ,
#                                          n_redundant = 0 ,
#                                          n_informative = 2,
#                                          n_datapts   = 200000,
#                                          n_classes   = 2,
#                                          random_state  = 47,
#                                          test_size   = 0.5)

data_train_ts_df, data_train_static_df, data_test_ts_df, data_test_static_df = generate_data_ts(data_dir = "",
                                        dataset_name = "synthetic_ts_1",
                                        n_features_static  = 2  ,
                                        n_features_ts      = 4  , 
                                        n_timepts   = 50,
                                        n_datapts   = 100, 
                                        test_size = 0.1,
                                        table_style = 'wide',
                                        n_classes   = 2)

# Load the Data

+ First import necessary stuff.
+ Look at simple implementations on how to handle the data
+ Then use ```Dataset``` class from Pytorch to handle the data.

In [2]:
import pandas as pd

Load the data and show the beginning.

In [3]:
path = '/home2/joshua.wendland/Documents/sepsis/toy_dataset/synthetic_ts_1/synthetic_ts_test_data_eav.csv.gz'

df = pd.read_csv(path, compression=None)
df = df.sort_values(by=['id', 'time'], ascending=True, ignore_index=True)  # time was not sorted

print(df.head)


<bound method NDFrame.head of         id  time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
0    id_90     0  0.627422 -0.478959 -0.892121 -0.318527  1.083295
1    id_90     1 -0.448220  0.019303 -0.127440 -0.157412  1.093168
2    id_90     2  0.637957 -0.386629 -0.416992  0.772078  1.026738
3    id_90     3  0.114574 -0.700368  0.415711 -0.584559  0.891013
4    id_90     4  0.318873  0.112828  0.575051  0.149870  0.813476
..     ...   ...       ...       ...       ...       ...       ...
495  id_99    45  0.033853  0.889838  0.867147 -0.226947  0.981591
496  id_99    46 -0.883889  1.506605  0.732750  0.466535  0.846787
497  id_99    47  1.267585 -0.623064  0.034502 -0.162447  0.901342
498  id_99    48 -1.544032  0.310212  1.469414  0.941757  0.912135
499  id_99    49  1.052585 -0.451113  1.395424 -0.229020  0.822956

[500 rows x 7 columns]>


Group the data according to each ```id``` of the time series. 
This is easily done by using the ```groupby()``` method on the DataFrame. This transforms the data into the structure of:

| Group Name (or ID) | Pandas Series   |  
| ------------------ | --------------- |
| id_0              | Series of id_0 |  
| id_1              | Series of id_1 |  
| ...              | ... |  

To receive a certain Series ID ```i``` we just need to call ```df_grouped[i, 0]```.
To receive the corresponding Series data we call ```df_grouped[i, 1]```.

In [110]:
# Group by 'id'
df_grouped = pd.DataFrame(df.groupby('id'))
# Get rid of the 'id'-column in each group
for x in df_grouped.iloc[:,1]:
    x.drop(columns=['id'], inplace=True)
    
print(df_grouped.iloc[-1,1].head())
print(df_grouped.iloc[-1,1].tail())

     time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
450     0  2.715997  0.325934 -0.486907 -2.543788  1.054109
451     1  1.526065  1.121756 -0.514032 -0.396498  0.764149
452     2 -0.505807  0.863921  0.798771 -0.444982  1.008917
453     3  0.788786  0.825285  0.303744 -1.426538  0.986686
454     4  0.219992 -0.188746 -0.861897 -0.611712  0.816993
     time    X_ts_0    X_ts_1    X_ts_2    X_ts_3      Y_ts
495    45  0.033853  0.889838  0.867147 -0.226947  0.981591
496    46 -0.883889  1.506605  0.732750  0.466535  0.846787
497    47  1.267585 -0.623064  0.034502 -0.162447  0.901342
498    48 -1.544032  0.310212  1.469414  0.941757  0.912135
499    49  1.052585 -0.451113  1.395424 -0.229020  0.822956


In [151]:
import numpy as np


X = df_grouped.drop(columns=0)
x = X.loc[1]
x

1        time    X_ts_0    X_ts_1    X_ts_2    X_ts...
Name: 1, dtype: object

## Using Pytorch class ```Dataset``` and ```Dataloader```

+ First import necessary modules.
+ Implement the ```Dataset``` class.

In [173]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

### `Dataset`

In [230]:
class ToyDataset(Dataset):
    """Dataset class for toy / synthetical data to test before continuing with EHR data. It is meant for multivariate time series.

            + `self.x_data`: Is a Pandas Dataframe, holding a list of time series, which also are Dataframes.
            + `self.x_ids`: Is a Pandas Dataframe, holding the corresponding id to each time series of `self.x_data`.

    Args:
        Dataset: The base Pytorch Dataset class.
    """

    def __init__(self, path):
        """Read in the data from *.csv file and prepare it. After init the class is left with self.x_data and self.x_ids.

        Args:
            path (str): Path to csv file.
        """
        # Read in csv
        df = pd.read_csv(path, compression=None)
        # Sort wrt id, then sort wrt time, ascending
        df = df.sort_values(by=['id', 'time'], ascending=True, ignore_index=True)
        # Number of samples
        self.n_samples = len(df['id'].unique())
        # Group by ID
        df_grouped = pd.DataFrame(df.groupby('id'))
        # Get rid of the 'id'-column in each group
        for x in df_grouped.iloc[:,1]:
            x.drop(columns=['id'], inplace=True)
        # extract the time series
        self.x_data = df_grouped.drop(columns=0)
        self.x_ids = df_grouped.drop(columns=1)

    def __getitem__(self, index):
        """Returns items from dataset for given index. Due to the nested structure of pd.DataFrames and np.arrays, some overhead instructions are necessary.

        Args:
            index (): Index for data sample(s).

        Returns:
            tuple: sample of data, id.
        """
        # Get the data from indexes
        X = self.x_data.loc[index]
        x_ids = self.x_ids.loc[index]
        x_ids = x_ids.tolist()
        # Extract, if multiple data, ...
        if isinstance(index, slice):
            x = list()
            for time_series in X[1]:
                # ... convert to numpy and append to list ...
                x.append(time_series.to_numpy())
            # ... and create one big Tensor array
            x = np.array(x) # first converting to numpy is much faster
            x = torch.tensor(x)
            # Create sample and return it
            sample = x, x_ids
        else:
            x = X.loc[1]
            x = torch.tensor(x.values)
            # x = x[None, :, :]  # same as unsequeeze
            sample = x, x_ids
            
        return sample


    def __len__(self):
        """The number of samples (time series) available in the dataset.

        Returns:
            int: Number of samples.
        """
        return self.n_samples

### `DataLoader`

Problems with Dataloader:
+ If there is an error and `num_workers!=0`, try setting it to `0`.
+ If there is an error and `shuffle=True`, try setting it to `False`. (Fixed)

In [237]:
dataset = ToyDataset(path)
train_loader = DataLoader(dataset=dataset, batch_size=3, shuffle=True, num_workers=0)

# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = next(dataiter)
features, labels = data
print( labels)

[('id_93', 'id_97', 'id_99')]
